In [6]:
import numpy as np
import pandas as pd
import nibabel as nib
import os

In [31]:

def create_region_df(region_files):
    df_rows = []

    # Process each region file
    for region_name, file_path in region_files.items():
        # Load the MRI data for the region
        region_data = np.array(nib.load(file_path).get_fdata(), dtype=np.int32)
        
        # Find unique voxel values and their indices
        unique_values, counts = np.unique(region_data, return_counts=True)
        
        # Iterate through each unique value to find its indices
        for value, count in zip(unique_values, counts):
            indices = list(zip(*np.where(region_data == value)))
            df_rows.append({
                "Region Name": region_name,
                "Region Unique Number": value,
                "Indices List of Voxels": indices,
                "Number of Voxels": count
            })
    
    # Create DataFrame
    region_df = pd.DataFrame(df_rows)
    return region_df



def create_and_save_masks(df, shape, output_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    sample_img = nib.load('/data/users3/jchen/atlas/Jean_abcd/abcd_mask/final/roiFrontal_forSmri.nii')
    affine = sample_img.affine

    # Create a new column for the mask file paths
    df['Mask File Path'] = ''
    
    
    df['Delta X'] = 0
    df['Delta Y'] = 0
    df['Delta Z'] = 0
    df['Mask Center'] = ''
    df['Mni_Scaled'] = ''

    # Iterate through each row in the DataFrame to create and save masks
    for index, row in df.iterrows():
        # Create an empty mask with the given shape
        mask = np.zeros(shape, dtype=np.uint8)
        
        # Set the indices to 1
        for coord in row['Indices List of Voxels']:
            mask[coord[0], coord[1], coord[2]] = 1
            
        indices = np.array(row['Indices List of Voxels'])
        
        
    ## Get maximum spans in each dimension
        df.at[index, 'Delta X'] = indices[:, 0].max() - indices[:, 0].min()
        df.at[index, 'Delta Y'] = indices[:, 1].max() - indices[:, 1].min()
        df.at[index, 'Delta Z'] = indices[:, 2].max() - indices[:, 2].min()
        
        
    ## Get the centre of the non-zero voxels.(Will be our manual postional encoding)
        non_zero_coords = np.argwhere(mask)
        min_coords = non_zero_coords.min(axis=0)
        max_coords = non_zero_coords.max(axis=0) + 1  # Add 1 because slice end index is exclusive
        
        # Compute the center of the bounding box
        center = [(max_coord + min_coord) // 2 for min_coord, max_coord in zip(min_coords, max_coords)]
        df.at[index,'Mask Center'] = center
        
        # Convert to MNI Scale
        center.append(1)
        voxel = np.array(center)
        mni_coords = affine.dot(voxel)[:3]
        mni_coords_scaled = [round(mni_coords[0]/75,3) ,  round(mni_coords[1]/110,3) , round(mni_coords[2]/85, 3)]
        df.at[index, 'Mni_Scaled'] = mni_coords_scaled
        
        # Define the file path for the mask
        file_name = f"{row['Region Name']}_{row['Region Unique Number']}.nii"
        file_path = os.path.join(output_dir, file_name)
        
        # Save the mask as a NIfTI file
        nib.save(nib.Nifti1Image(mask, affine), file_path)
        
        # Update the DataFrame with the mask file path
        df.at[index, 'Mask File Path'] = file_path
        
    df = df.drop('Indices List of Voxels', axis = 1)

    return df



In [32]:


# Define file paths for each region
region_files = {
    "frontal": '/data/users3/jchen/atlas/Jean_abcd/abcd_mask/final/roiFrontal_forSmri.nii',
    "cereb": '/data/users3/jchen/atlas/Jean_abcd/abcd_mask/final/roiCerebellum_forSmri.nii',
    "RN": '/data/users3/jchen/atlas/Jean_abcd/abcd_mask/final/roiRN_forSmri.nii',
    "thalamus": '/data/users3/jchen/atlas/Jean_abcd/abcd_mask/final/roiThalamus_forSmri.nii',
    "parietal": '/data/users3/jchen/atlas/Jean_abcd/abcd_mask/final/roiParietal_forSmri.nii'
}

# Generate the DataFrame
region_df = create_region_df(region_files)
# Filter out rows where 'Region Unique Number' is 0
filtered_df = region_df[region_df['Region Unique Number'] != 0]

# Assuming 'region_df' is your DataFrame created earlier and filtered
updated_df1 = create_and_save_masks(filtered_df, shape = (121,145,121), output_dir = '/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_Mask_Data/masks_3D_parietal')

/tmp/ipykernel_44763/981103537.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Mask File Path'] = ''
/tmp/ipykernel_44763/981103537.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Delta X'] = 0
/tmp/ipykernel_44763/981103537.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [36]:
updated_df1

,Region Name,Region Unique Number,Number of Voxels,Mask File Path,Delta X,Delta Y,Delta Z,Mask Center,Mni_Scaled
1,frontal,32,51,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,14,18,10,"[93, 79, 49, 1]","[-0.66, -0.068, 0.018]"
2,frontal,33,22,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,10,7,4,"[97, 75, 54, 1]","[-0.74, -0.123, 0.106]"
3,frontal,34,105,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,8,5,10,"[85, 70, 54, 1]","[-0.5, -0.191, 0.106]"
4,frontal,35,492,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,7,14,12,"[87, 78, 55, 1]","[-0.54, -0.082, 0.124]"
5,frontal,36,83,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,8,6,4,"[85, 70, 59, 1]","[-0.5, -0.191, 0.194]"
...,...,...,...,...,...,...,...,...,...
332,parietal,362,125,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,11,7,5,"[25, 55, 65, 1]","[0.7, -0.395, 0.3]"
333,parietal,363,382,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,15,9,8,"[27, 44, 66, 1]","[0.66, -0.545, 0.318]"
334,parietal,364,828,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,17,11,10,"[26, 49, 67, 1]","[0.68, -0.477, 0.335]"
335,parietal,365,560,/data/users4/sdeshpande8/3D_Mask_Brain_CNN/3D_...,14,9,13,"[22, 54, 70, 1]","[0.76, -0.409, 0.388]"


In [37]:
updated_df1.to_csv('3D_Masks_Data_parietal.csv', index=False)